In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Input Data Description
1. The origins of your data. Where did you get the data? How were the data collected from the original sources?  
2. The structure, granularity, scope, temporality and faithfulness (SGSTF) of your data. To discuss these attributes you should load the data into one or more data frames (so you'll start building code cells for the first time). At a minimum, use some basic methods (.head, .loc, and so on) to provide support for the descriptions you provide for SGSTF.

This dataset is from [nu3](https://www.nu3.de/blogs/nutrition/food-carbon-footprint-index-2018), a German online personal wellness and grocery store that performed some research to look at the CO2 emissions per person for 130 nations worldwide. Their study utilized 2018 data from the Food and Agriculture Organization of the United Nations (FAO) that described the amount of food supply for consumption by food item and nation. They also used the following assumptions as the estimated CO2 emissions per kilogram for each food type in 2018:

- Pork = 3.54 kg
- Poultry= 1.07 kg
- Beef = 30.86 kg
- Lamb & Goat = 35.02 kg
- Fish = 1.60 kg
- Eggs = 0.92 kg
- Milk – including cheese = 1.42 kg
- Wheat & Wheat Products = 0.19 kg
- Rice = 1.28 kg
- Soybeans = 0.45 kg
- Nuts – including Peanut Butter = 1.77 kg

### Data Acquisition and SGSTF

First, we grab the raw html from the website, and clean up the data to put it in a readable format.

In [13]:
# https://www.nu3.de/blogs/nutrition/food-carbon-footprint-index-2018
food = pd.read_html("https://www.nu3.de/blogs/nutrition/food-carbon-footprint-index-2018")[0]
food = food.droplevel([0,1], axis=1)
columns = ['#', 'Country', 'porksupply', 'porkco2', 'chickensupply', 'chickenco2', 'beefsupply', 'beefco2', 'lambsupply', 'lambco2', 
           'fishsupply', 'fishco2', 'eggsupply', 'eggco2', 'milkcheesesupply', 'milkcheeseco2', 'totalanimalco2', 
           'wheatprodsupply', 'wheatprodco2', 'ricesupply', 'riceco2', 'soybeansupply', 'soybeanco2', 
           'nutssupply', 'nutsco2', 'totalnonanimalco2', 'difftotalco2']

food.rename(columns=dict(zip(food.columns, columns)), inplace=True)
food.head()

,#,Country,porksupply,porkco2,chickensupply,chickenco2,beefsupply,beefco2,lambsupply,lambco2,...,wheatprodsupply,wheatprodco2,ricesupply,riceco2,soybeansupply,soybeanco2,nutssupply,nutsco2,totalnonanimalco2,difftotalco2
0,1,Argentina,10.51,37.20,38.66,41.53,55.48,1712.00,1.56,54.63,...,103.11,19.66,8.77,11.22,0.00,0.00,0.49,0.87,31.75,2108.90
1,2,Australia,24.14,85.44,46.12,49.54,33.86,1044.85,9.87,345.65,...,70.46,13.44,11.03,14.12,0.19,0.09,8.73,15.45,43.09,1852.46
2,3,Albania,10.88,38.51,13.23,14.21,22.50,694.30,15.32,536.50,...,138.64,26.44,7.78,9.96,0.00,0.00,4.36,7.72,44.11,1689.62
3,4,Iceland,21.69,76.77,26.87,28.86,13.36,412.26,21.12,739.62,...,72.92,13.91,3.89,4.98,0.11,0.05,3.88,6.87,25.80,1679.75
4,5,New Zealand,22.29,78.90,34.98,37.58,22.49,693.99,18.91,662.23,...,76.91,14.67,9.16,11.72,0.44,0.20,8.22,14.55,41.14,1668.67


`Structure:`  
`Granularity:`  
`Scope:`  
`Temporality:`  
`Faithfulness:`  

In [3]:
# 130 different countries
food.shape

(130, 27)

---

## Data Cleaning

First, I constructed the Y variable, Total Kilograms CO2 Emissions / person / year, as it wasn't present in the original dataset, but we had the individual totals of animal products and non-animal products. Then, we can remove the other total columns, since we don't need them anymore.

In [4]:
food['totalco2'] = food['totalanimalco2'] + food['totalnonanimalco2']
food.drop(columns=['#', 'totalanimalco2', 'totalnonanimalco2', 'difftotalco2'], inplace=True)
food.head()

,Country,porksupply,porkco2,chickensupply,chickenco2,beefsupply,beefco2,lambsupply,lambco2,fishsupply,...,milkcheeseco2,wheatprodsupply,wheatprodco2,ricesupply,riceco2,soybeansupply,soybeanco2,nutssupply,nutsco2,totalco2
0,Argentina,10.51,37.20,38.66,41.53,55.48,1712.00,1.56,54.63,4.36,...,277.87,103.11,19.66,8.77,11.22,0.00,0.00,0.49,0.87,2172.40
1,Australia,24.14,85.44,46.12,49.54,33.86,1044.85,9.87,345.65,17.69,...,334.01,70.46,13.44,11.03,14.12,0.19,0.09,8.73,15.45,1938.64
2,Albania,10.88,38.51,13.23,14.21,22.50,694.30,15.32,536.50,3.85,...,432.62,138.64,26.44,7.78,9.96,0.00,0.00,4.36,7.72,1777.84
3,Iceland,21.69,76.77,26.87,28.86,13.36,412.26,21.12,739.62,74.41,...,321.66,72.92,13.91,3.89,4.98,0.11,0.05,3.88,6.87,1731.35
4,New Zealand,22.29,78.90,34.98,37.58,22.49,693.99,18.91,662.23,20.36,...,195.50,76.91,14.67,9.16,11.72,0.44,0.20,8.22,14.55,1750.94


Then, I decided to separate the data into the CO2 columns and the Food Supply columns. This is because our Y variable is fully described by the other CO2 columns, so using them in our model would lead to a trivial problem to solve.

In [5]:
[col for col in food.columns if re.findall('supply', col)]

['porksupply',
 'chickensupply',
 'beefsupply',
 'lambsupply',
 'fishsupply',
 'eggsupply',
 'milkcheesesupply',
 'wheatprodsupply',
 'ricesupply',
 'soybeansupply',
 'nutssupply']

In [6]:
food_co2 = food.drop(columns=[col for col in food.columns if re.findall('supply', col)])
food_co2.head()

,Country,porkco2,chickenco2,beefco2,lambco2,fishco2,eggco2,milkcheeseco2,wheatprodco2,riceco2,soybeanco2,nutsco2,totalco2
0,Argentina,37.20,41.53,1712.00,54.63,6.96,10.46,277.87,19.66,11.22,0.00,0.87,2172.40
1,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,0.09,15.45,1938.64
2,Albania,38.51,14.21,694.30,536.50,6.15,11.44,432.62,26.44,9.96,0.00,7.72,1777.84
3,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,0.05,6.87,1731.35
4,New Zealand,78.90,37.58,693.99,662.23,32.51,9.10,195.50,14.67,11.72,0.20,14.55,1750.94


In [7]:
[col for col in food.columns if re.findall('co2', col)][:-1]

['porkco2',
 'chickenco2',
 'beefco2',
 'lambco2',
 'fishco2',
 'eggco2',
 'milkcheeseco2',
 'wheatprodco2',
 'riceco2',
 'soybeanco2',
 'nutsco2']

In [8]:
food_supply = food.drop(columns=[col for col in food.columns if re.findall('co2', col)][:-1])
food_supply.head()

,Country,porksupply,chickensupply,beefsupply,lambsupply,fishsupply,eggsupply,milkcheesesupply,wheatprodsupply,ricesupply,soybeansupply,nutssupply,totalco2
0,Argentina,10.51,38.66,55.48,1.56,4.36,11.39,195.08,103.11,8.77,0.00,0.49,2172.40
1,Australia,24.14,46.12,33.86,9.87,17.69,8.51,234.49,70.46,11.03,0.19,8.73,1938.64
2,Albania,10.88,13.23,22.50,15.32,3.85,12.45,303.72,138.64,7.78,0.00,4.36,1777.84
3,Iceland,21.69,26.87,13.36,21.12,74.41,8.24,225.82,72.92,3.89,0.11,3.88,1731.35
4,New Zealand,22.29,34.98,22.49,18.91,20.36,9.91,137.25,76.91,9.16,0.44,8.22,1750.94


In [9]:
# check that the totalco2 is roughly the sum of the co2 columns by looking at the largest difference between the two
max(food_co2.loc[:, food_co2.columns[1:-1]].values.sum(axis=1) - food_co2['totalco2'].values)

0.029999999999972715

### Motivation for conversion of data to percentages:
We converted the supply columns to percentages so that it is more comparable to an individual's diet. People tend to have a *better grasp of the relative amounts of each food group they eat rather than the raw amount*, which is what the data was before cleaning. That way, our model can take inputs of the percent of each food group an individual consumes on an annual basis, and relate that directly to `totalco2`.

In [10]:
food_supply.columns[1:-2]

Index(['porksupply', 'chickensupply', 'beefsupply', 'lambsupply', 'fishsupply',
       'eggsupply', 'milkcheesesupply', 'wheatprodsupply', 'ricesupply',
       'soybeansupply'],
      dtype='object')

In [11]:
# create a column of the total supply of each row
food_supply['totalsupply'] = food_supply.loc[:, columns_supply].values.sum(axis=1)
columns_supply = food_supply.columns[1:-2] # ignore the Country, totalco2, totalsupply columns

# convert columns into percentages
food_pct = pd.DataFrame([])
for col in columns_supply:
    food_pct[col + '_pct'] = food_supply[col] / food_supply['totalsupply']
    
# add the y variable and country to the new df
food_pct['totalco2'] = food_co2['totalco2']
food_pct['Country'] = food['Country']
food_pct.head()

NameError: name 'columns_supply' is not defined

In [ ]:
# food_pct.to_csv('../../data/nu3-pctfoodsupply-CO2-emissions.csv')

---

## Exploratory Data Analysis

We now have one main dataset that will be used for modelling: `food_pct`.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.heatmap(food_pct.corr(), annot=True, vmin=-1, ax=ax)
plt.title('Correlation Matrix');

We see that certain pairings of our features have strong correlation:
- rice supply & milkcheese supply: -0.75
- fish supply & milkcheese supply: -0.6
- pork supply & egg supply: 0.41

Certain features also have stronger correlation with our y variable `totalco2`:
- milkcheese supply: 0.54
- rice supply: -0.49
- beef supply: 0.45

But nothing particularly interesting, or any immediate reason that these variables would have the noted correlations. We can keep this matrix in mind when performing feature selection, as we want to prevent collinearity and improve our AIC/Adj. $R^2$ by keeping our chosen covariates distinct.

In [ ]:
plt.figure(figsize=(10,6))

plt.subplot(2,2,1)
plt.hist(food_pct['totalco2'], bins = np.linspace(food_pct['totalco2'].min(), food_pct['totalco2'].max(), 16));
plt.title('Distribution of Y Variable: Total CO2')
plt.xlabel('Total CO2')
plt.yticks(np.arange(0, 21, 2))
plt.ylabel('Counts')

plt.subplot(2,2,2)
plt.hist(np.log(food_pct['totalco2']), bins = np.linspace(np.log(food_pct['totalco2']).min(), 
                                                            np.log(food_pct['totalco2']).max(), 16));
plt.title('Distribution of Y Variable: log(Total KG CO2)/person/year')
plt.xlabel('log(Total CO2)')
plt.yticks(np.arange(0, 21, 2))
plt.ylabel('Counts')

plt.subplot(2,2,3)
sns.boxplot(food_pct['totalco2'])
plt.xlabel('Total KG CO2 /person/year')
plt.title('Boxplot of Y Variable')

plt.subplot(2,2,4)
sns.boxplot(np.log(food_pct['totalco2']))
plt.xlabel('log(Total KG CO2) /person/year')
plt.title('Boxplot of log(Y Variable)')

plt.tight_layout()

Our Y variable is right skewed, but is not so strong where we need a log transformation for our model. We may try using transformations to improve our model later on, but for now, we will move forward without any variable transformations.

Here is the distribution of the **average American diet** according to our data.

In [ ]:
names = ['Pork', 'Chicken', 'Beef', 'Lamb', 'Fish', 'Eggs', 'Milk&Cheese', 'Wheat', 'Rice', 'Soybeans']
usa = food_pct[food_pct['Country'] == 'USA'].drop(columns=['Country', 'totalco2']).rename(columns=dict(zip(food_pct.columns[:-1], names)))
usa

In [ ]:
plt.figure(figsize=(12,4))
plt.bar(usa.columns, usa.to_numpy()[0])
plt.title('Average American Diet of Included Products')
plt.ylabel('% of Diet in Weight');

---

Even though we are not looking at country location in our model, we thought it'd be interesting to plot it spatially and see if there are any trends or clusters that have higher kgCO2/person/year.

Using this [guide](https://towardsdatascience.com/using-python-to-create-a-world-map-from-a-list-of-country-names-cd7480d03b10) as a basis, we plot a world map of the KG CO2 emissions per person.

In [ ]:
country_co2 = food_pct.loc[:, ['totalco2', 'Country']]
country_co2.rename(columns = {'Country' : 'CountryName'}, inplace=True)
country_co2.head()

In [ ]:
#installation
# !pip install pycountry-convert

#function to convert to alpha2 country codes and continents
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    
    return (cn_a2_code, cn_continent)

In [ ]:
codes = country_co2['CountryName'].apply(get_continent)

country_co2['Country'] = [code[0] for code in codes]
country_co2['Continent'] = [code[1] for code in codes]
country_co2.head()

In [ ]:
#installation
# !pip install geopy

#function to get longitude and latitude data from country name
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="nu3-data-analysis")
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return (np.nan, np.nan)

In [ ]:
latlons = country_co2['Country'].apply(geolocate)

country_co2['Latitude'] = [latlon[0] for latlon in latlons]
country_co2['Longitude'] = [latlon[1] for latlon in latlons]
country_co2.head()

In [ ]:
country_co2.loc[country_co2['Latitude'].isna(), :]

In [ ]:
# temporarily removing them to test the world map
country_co2_clean = country_co2.loc[~country_co2['Latitude'].isna(), :]

In [ ]:
#installation
# !pip install folium

# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster

#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(country_co2_clean)):
        lat = country_co2_clean.iloc[i]['Latitude']
        long = country_co2_clean.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    kgCO2/person/year : {}<br>"""
        popup_text = popup_text.format(country_co2_clean.iloc[i]['CountryName'],
                                   country_co2_clean.iloc[i]['totalco2']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

#### Another map:

In [ ]:
# !pip install country-converter

import country_converter as coco

country_co2['CountryCode'] = coco.convert(names=country_co2['Country'].to_list(), to='ISO3') # ignore the error

In [ ]:
import plotly.express as px

fig = px.choropleth(country_co2, locations="CountryCode",
                    color="totalco2", 
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
from  import function